In [ ]:
import tensorflow as tf
from tensorflow import keras
import time
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot  as plt

def _save_img2img_gan( ep, img1, img2,seed,loss_type):
    if not isinstance(img1, np.ndarray):
        img1 = img1.numpy()
    if not isinstance(img2, np.ndarray):
        img2 = img2.numpy()
    if img1.ndim > 3:
        img1 = np.squeeze(img1, axis=-1)
    if img2.ndim > 3:
        img2 = np.squeeze(img2, axis=-1)
    img1, img2 = _img_recenter(img1), _img_recenter(img2)
    plt.clf()
    nc, nr = 20, 10
    plt.figure(0, (nc * 2, nr * 2))
    i = 0
    for c in range(0, nc, 2):
        for r in range(nr):
            n = r * nc + c
            plt.subplot(nr, nc, n + 1)
            plt.imshow(img1[i], cmap="gray")
            plt.axis("off")
            plt.subplot(nr, nc, n + 2)
            plt.imshow(img2[i], cmap="gray_r")
            plt.axis("off")
            i += 1

    plt.tight_layout()
    path_='drive/Shared drives/Ziqiang/Mnist_condi/For_github/'+loss_type+'/'+seed
    dir_ = path_+'/visual_mnist'
    os.makedirs(dir_, exist_ok=True)
    path = dir_ +'/{}.png'.format(ep)
    plt.savefig(path)
    # plt.close()
    plt.clf()

def _img_recenter(img):
    return (img + 1) * 255 / 2

def save_gan(model, ep, **kwargs):
    name = model.__class__.__name__.lower()
    if name == "cyclegan":
        if "img6" not in kwargs or "img9" not in kwargs:
            raise ValueError

        #for t-sne
        img6, img9 = kwargs["img6"], kwargs["img9"]
        y6, y9 = kwargs["y6"], kwargs["y9"]
        img9_, img6_ = model.g12.call([img6,y9], training=False), model.g21.call([img9,y6], training=False)
        x_eval=np.vstack((img9_.numpy(),img9.numpy())).reshape(-1,28*28*1)
        num=100
        y_eval=np.array([0]*num+[1]*num+[2]*num+[3]*num+[4]*num+[5]*num+[6]*num+[7]*num+[8]*num
                        +[9]*num+[10]*num+[11]*num+[12]*num+[13]*num+[14]*num+[15]*num+[16]*num+[17]*num)
        #MSE
        score6=tf.reduce_mean(tf.losses.mean_squared_error(img9,img9_))
        
        score=score6

        #visual MNIST
        img1, img2 = kwargs["img6"][:50], kwargs["img9"][400:450]
        y1, y2 = kwargs["y6"][:50], kwargs["y9"][400:450]
        img2_, img1_ = model.g12.call([img1,y2], training=False), model.g21.call([img2,y1], training=False)
        img = np.concatenate((img1.numpy(), img2.numpy()), axis=0)
        imgs = np.concatenate((img2_.numpy(), img1_.numpy()), axis=0)
        
        return score.numpy(),x_eval,y_eval,img,imgs

    else:
        raise ValueError(name)
    # plt.clf()
    # plt.close()
